# エネルギーの需要予測モデリング & モデル解釈 (リモート高速実行)

過去のエネルギー需要実績データから、将来必要なエネルギー需要を予測します。<br>
※参考 : [時系列予測モデルを自動トレーニングする](https://docs.microsoft.com/ja-JP/azure/machine-learning/service/how-to-auto-train-forecast)


1. 事前準備
    - Python SDK のインポート
    - Azure ML `Workspace` への接続
    - `Experiment` の作成
    - `Dataset` の作成と登録


2. 自動機械学習 Automated Machine Learning
    - 計算環境 `Machine Learning Compute` の準備
    - 自動機械学習 Automated ML の事前設定
    - モデル学習と結果の確認


3. モデル解釈

## 1. 事前準備

### Python SDK のインポート
Azure Machine Learning の Python SDK をインポートします。

In [1]:
import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime

Azure Machine Learning Python SDK のバージョンを確認します。

In [2]:
print(azureml.core.VERSION)

1.0.74


In [3]:
# その他必要なライブラリをインポートします。
import logging
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# 警告メッセージを削除する
warnings.showwarning = lambda *args, **kwargs: None

### Azure ML workspace との接続
Azure Machine Learning との接続を行います。Azure Active Directory の認証が必要です。

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定
Azure Machine Learing では 実験を管理する仕組みがあります。自動機械学習は自動的にその実験管理の仕組みでメトリックやログが残ります。

In [5]:
# choose a name for experiment
experiment_name = 'automl-forecasting-energydemand'
experiment = Experiment(ws, experiment_name)

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning service の Dataset のフォーマットでデータを定義します。

In [6]:
# 予測対象変数と日時カラムの指定
label = 'demand' 
time_column_name = 'timeStamp'

In [7]:
dataset = Dataset.Tabular.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/nyc_energy.csv").with_timestamp_columns(fine_grain_timestamp=time_column_name) 
dataset.take(5).to_pandas_dataframe()

,timeStamp,demand,precip,temp
0,2012-01-01 00:00:00,4937.50,0.00,46.13
1,2012-01-01 01:00:00,4752.10,0.00,45.89
2,2012-01-01 02:00:00,4542.60,0.00,45.04
3,2012-01-01 03:00:00,4357.70,0.00,45.03
4,2012-01-01 04:00:00,4275.50,0.00,42.61


In [8]:
# Cut off the end of the dataset due to large number of nan values
dataset = dataset.time_before(datetime(2017, 10, 10, 5))

In [ ]:
# 2017年8月8日5:00 までを学習データ
train_dataset = dataset.time_before(datetime(2017, 8, 8, 5), include_boundary=True)
train_dataset.to_pandas_dataframe().sort_values(time_column_name).tail(5)

,timeStamp,demand,precip,temp
49071,2017-08-08 01:00:00,5106.35,0.00,68.30
49072,2017-08-08 02:00:00,4947.73,0.00,68.44
49073,2017-08-08 03:00:00,4867.02,0.00,68.78
49074,2017-08-08 04:00:00,4888.20,0.00,68.70
49075,2017-08-08 05:00:00,5120.31,0.00,67.53


In [ ]:
# 2017年8月8日5:00 - 2017年8月10日5:00 までをテストデータ
test_dataset = dataset.time_between(datetime(2017, 8, 8, 5), datetime(2017, 8, 10, 5))
test_dataset.to_pandas_dataframe().head(5)

,timeStamp,demand,precip,temp
0,2017-08-08 06:00:00,5590.99,0.00,66.17
1,2017-08-08 07:00:00,6147.03,0.00,66.29
2,2017-08-08 08:00:00,6592.43,0.00,66.72
3,2017-08-08 09:00:00,6874.53,0.00,67.37
4,2017-08-08 10:00:00,7010.54,0.00,68.30


### 計算環境 Machine Learning Compute の設定

Azure Machine Learning では機械学習のための計算環境 Machine Learning Compute が利用できます。

In [ ]:
# 予め Azure ML studio にて cpucluster という名称の Machine Learning Compute を 作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(ws, name = "cpucluster")

## 2. 自動機械学習 Automated Machine Learning
自動機械学習の事前設定を行います。[AutoMLConfig](https://docs.microsoft.com/ja-JP/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) を用いて定義します。

In [ ]:
max_horizon = 48

In [ ]:
automl_settings = {
    'time_column_name': time_column_name,
    "max_concurrent_iterations": 4,
    'max_horizon': max_horizon,
}

automl_config = AutoMLConfig(task='forecasting',                             
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=10,
                             iterations = 5,
                             training_data=train_dataset,
                             label_column_name=label,
                             #compute_target=compute_target,  # ローカル環境で実行する場合には compute_target, max_concurrent_itterations をコメントアウトします
                             n_cross_validations=3, 
                             model_explainability = True,
                             verbosity=logging.INFO,
                            **automl_settings)

### 実行と結果確認

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_98740e4c-629a-4213-b80b-1687a96ccdf4
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summ

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
automl_run, fitted_model = remote_run.get_output()

### モデルの理解

In [ ]:
#pd.DataFrame(fitted_model.named_steps['timeseriestransformer'].get_engineered_feature_names())

In [ ]:
#pd.DataFrame.from_records(fitted_model.named_steps['timeseriestransformer'].get_featurization_summary())

## 3. モデル解釈
Azure Machine Learning には Automated ML のモデルを解釈する仕組みがあります。詳しくは [モデルを解釈する方法](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-machine-learning-interpretability#how-to-interpret-your-model)をご参照ください。

In [ ]:
# Pandas Dataframe に変換
train_df = train_dataset.to_pandas_dataframe()
test_df = test_dataset.to_pandas_dataframe()

In [ ]:
from azureml.train.automl.runtime.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations
automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, 
                                                             X=train_df.drop([label], axis=1), 
                                                             X_test=test_df.drop([label], axis=1), 
                                                             y=train_df[label].values, 
                                                             task='forecasting')

In [ ]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel,
                         init_dataset=automl_explainer_setup_obj.X_transform, run=automl_run,
                         features=automl_explainer_setup_obj.engineered_feature_names,
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

## Engineered Explanation
特徴量エンジニアリングで生成された変数を用いたモデル解釈を行います。

In [ ]:
# Compute the engineered explanations
engineered_explanations = explainer.explain(['local', 'global'],get_raw=False,
                                            eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(engineered_explanations, 
                     automl_explainer_setup_obj.automl_estimator, 
                     datasetX=automl_explainer_setup_obj.X_test_transform)

## Raw Explanation
特徴量エンジニアリング前の変数を用いたモデル解釈を行います。

In [ ]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(raw_explanations, 
                     automl_explainer_setup_obj.automl_pipeline, 
                     datasetX=automl_explainer_setup_obj.X_test_raw)